In [9]:
import torch
torch.cuda.is_available()

True

In [10]:
torch.rand(10).cuda()
#conda install pytorch==1.10.0 torchvision==0.11.0 torchaudio==0.10.0 cudatoolkit=11.3 -c pytorch -c conda-forge
#pip install mayavi
#install cuda 11.3 version
#python setup.py develop
#conda install opencv
#pip install spconv-cu113
#pip install open3d filterpy
# !pip install motmetrics


tensor([0.6049, 0.2849, 0.8211, 0.1337, 0.8801, 0.3372, 0.0885, 0.7971, 0.7936,
        0.1552], device='cuda:0')

In [11]:
#conda install numba
import argparse
import glob
from pathlib import Path

try:
    import open3d
    from visual_utils import open3d_vis_utils as V
    OPEN3D_FLAG = True
except:
    import mayavi.mlab as mlab
    from visual_utils import visualize_utils as V
    OPEN3D_FLAG = False

import numpy as np
import torch

from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.datasets import DatasetTemplate
from pcdet.models import build_network, load_data_to_gpu
from pcdet.utils import common_utils


In [12]:


class DemoDataset(DatasetTemplate):
    def __init__(self, dataset_cfg, class_names, training=True, root_path=None, logger=None, ext='.bin'):
        """
        Args:
            root_path:
            dataset_cfg:
            class_names:
            training:
            logger:
        """
        super().__init__(
            dataset_cfg=dataset_cfg, class_names=class_names, training=training, root_path=root_path, logger=logger
        )
        self.root_path = root_path
        self.ext = ext
        data_file_list = glob.glob(str(root_path / f'*{self.ext}')) if self.root_path.is_dir() else [self.root_path]

        data_file_list.sort()
        self.sample_file_list = data_file_list

    def __len__(self):
        return len(self.sample_file_list)

    def __getitem__(self, index):
        if self.ext == '.bin':
            points = np.fromfile(self.sample_file_list[index], dtype=np.float32).reshape(-1, 4)
        elif self.ext == '.npy':
            points = np.load(self.sample_file_list[index])
        else:
            raise NotImplementedError

        input_dict = {
            'points': points,
            'frame_id': index,
        }

        data_dict = self.prepare_data(data_dict=input_dict)
        return data_dict


In [13]:
def parse_config():
    parser = argparse.ArgumentParser(description='arg parser')
    parser.add_argument('--cfg_file', type=str, default='cfgs/kitti_models/pv_rcnn.yaml',
                        help='specify the config for demo')
    parser.add_argument('--data_path', type=str, default='demo_data',
                        help='specify the point cloud data file or directory')
    parser.add_argument('--ckpt', default = "checkpoints/pv_rcnn_8369.pth",type=str,  help='specify the pretrained model')
    parser.add_argument('--ext', type=str, default='.bin', help='specify the extension of your point cloud data file')

    args = parser.parse_args(args=[])

    cfg_from_yaml_file(args.cfg_file, cfg)

    return args, cfg


In [14]:
args, cfg = parse_config()
logger = common_utils.create_logger()
logger.info('-----------------Quick Demo of OpenPCDet-------------------------')

2023-02-16 14:43:50,399   INFO  -----------------Quick Demo of OpenPCDet-------------------------
2023-02-16 14:43:50,399   INFO  -----------------Quick Demo of OpenPCDet-------------------------


In [15]:
demo_dataset = DemoDataset(
    dataset_cfg=cfg.DATA_CONFIG, class_names=cfg.CLASS_NAMES, training=False,
    root_path=Path(args.data_path), ext=args.ext, logger=logger
)
# logger.info(f'Total number of samples: \t{len(demo_dataset)}')


In [16]:
#only car 
def apply_thres(pred_dicts,thres=0.5):
    detected_bbox_list = []
    label_list =[]
    ref_scores_list = []
    for idx,bbox in enumerate(pred_dicts[0]['pred_boxes']):
        if thres > pred_dicts[0]['pred_scores'][idx] or pred_dicts[0]['pred_labels'][idx] != 1:
            continue
        detected_bbox_list.append(bbox.tolist())
        ref_scores_list.append(pred_dicts[0]['pred_scores'][idx])
        label_list.append(pred_dicts[0]['pred_labels'][idx])
    return detected_bbox_list,ref_scores_list,label_list
# print(detected_bbox)

In [17]:
#TODO : use mayavi
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=demo_dataset)
model.load_params_from_file(filename=args.ckpt, logger=logger, to_cpu=True)
model.cuda()
model.eval()
pred_det_list=[]
points_list=[]
ref_scores_list=[]
ref_label_list = []

with torch.no_grad():
    for idx, data_dict in enumerate(demo_dataset):
        data_dict = demo_dataset.collate_batch([data_dict])
        load_data_to_gpu(data_dict)
        points_list.append(data_dict['points'][:, 1:])
        pred_dicts, _ = model.forward(data_dict)
        filtered_det,score,label = apply_thres(pred_dicts)
        pred_det_list.append(filtered_det)
        ref_scores_list.append(score)
        ref_label_list.append(label)


/home/server-003/anaconda3/envs/pcdet/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272068694/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2023-02-16 14:43:52,780   INFO  ==> Loading parameters from checkpoint checkpoints/pv_rcnn_8369.pth to CPU
2023-02-16 14:43:52,780   INFO  ==> Loading parameters from checkpoint checkpoints/pv_rcnn_8369.pth to CPU
2023-02-16 14:43:52,821   INFO  ==> Done (loaded 367/367)
2023-02-16 14:43:52,821   INFO  ==> Done (loaded 367/367)


In [18]:
def translate_boxes_to_open3d_instance(gt_boxes):
    """
             4-------- 6
           /|         /|
          5 -------- 3 .
          | |        | |
          . 7 -------- 1
          |/         |/
          2 -------- 0
    """
    center = gt_boxes[0:3]
    lwh = gt_boxes[3:6]
    axis_angles = np.array([0, 0, gt_boxes[6] + 1e-10])
    rot = open3d.geometry.get_rotation_matrix_from_axis_angle(axis_angles)
    box3d = open3d.geometry.OrientedBoundingBox(center, rot, lwh)

    line_set = open3d.geometry.LineSet.create_from_oriented_bounding_box(box3d)

    # import ipdb; ipdb.set_trace(context=20)
    lines = np.asarray(line_set.lines)
    lines = np.concatenate([lines, np.array([[1, 4], [7, 6]])], axis=0)

    line_set.lines = open3d.utility.Vector2iVector(lines)

    return line_set, box3d

In [19]:
# [[h,w,l,x,y,z,theta],...]
#  bbox.h, bbox.w, bbox.l, bbox.x, bbox.y, bbox.z, bbox.ry = data[:7]
"""
    7 -------- 4
   /|         /|
  6 -------- 5 .
  | |        | |
  . 3 -------- 0
  |/         |/
  2 -------- 1
Args:
    boxes3d:  (N, 7) [x, y, z, dx, dy, dz, heading], (x, y, z) is the box center

Returns:
"""

'\n    7 -------- 4\n   /|         /|\n  6 -------- 5 .\n  | |        | |\n  . 3 -------- 0\n  |/         |/\n  2 -------- 1\nArgs:\n    boxes3d:  (N, 7) [x, y, z, dx, dy, dz, heading], (x, y, z) is the box center\n\nReturns:\n'

In [20]:
import os, numpy as np, time, sys, argparse
from tracking_modules.utils import Config, get_subfolder_seq, createFolder
from tracking_modules.io import (
    load_detection,
    get_saving_dir,
    get_frame_det,
    save_results,
    save_affinity,
)
from tracking_modules.model import AB3DMOT


In [21]:
config_path = "./tracking_modules/configs/config.yml"
cfg = Config(config_path)

In [22]:
import numpy as np
import open3d as o3d
def text_3d(text, pos, direction=None, degree=0.0,density=5, font="/usr/share/fonts/truetype/dejavu/DejaVuSans-BoldOblique.ttf", font_size=210):
    """
    Generate a 3D text point cloud used for visualization.
    :param text: content of the text
    :param pos: 3D xyz position of the text upper left corner
    :param direction: 3D normalized direction of where the text faces
    :param degree: in plane rotation of text
    :param font: Name of the font - change it according to your system
    :param font_size: size of the font
    :return: o3d.geoemtry.PointCloud object
    """
    if direction is None:
        direction = (0, 0., 1.)

    from PIL import Image, ImageFont, ImageDraw
    from pyquaternion import Quaternion
#     font_obj = ImageFont.load_default()
    font_obj = ImageFont.truetype(font, font_size*density)

    font_dim = font_obj.getsize(text)

    img = Image.new('RGB', font_dim, color=(255, 255, 255))
    draw = ImageDraw.Draw(img)
    draw.text((0, 0), text, font=font_obj, fill=(0, 0, 0))
    img = np.asarray(img)
    img_mask = img[:, :, 0] < 128
    indices = np.indices([*img.shape[0:2], 1])[:, img_mask, 0].reshape(3, -1).T

    pcd = o3d.geometry.PointCloud()
    pcd.colors = o3d.utility.Vector3dVector(img[img_mask, :].astype(float)+100 )
#     / 255.0
    pcd.points = o3d.utility.Vector3dVector(indices / 1000 /density)

    raxis = np.cross([0.0, 0.0, 1.0], direction)
    if np.linalg.norm(raxis) < 1e-6:
        raxis = (0.0, 0.0, 1.0)
    trans = (Quaternion(axis=raxis, radians=np.arccos(direction[2])) *
             Quaternion(axis=direction, degrees=degree)).transformation_matrix
    trans[0:3, 3] = np.asarray(pos)
    pcd.transform(trans)
    return pcd
# text_3d("test",[0,0,0])

In [23]:
gt_path = "/home/server-003/workspace/dataset/kitti_tracking/data_tracking_label_2/training/label_02/0000.txt"
gt_id_list=[]
gt_bbox_list=[]
#TODO : cam_to_lidar plane, class 
# gt_frame_list=[]
with open(gt_path, "r") as f:
    file = f.readlines()
#     print(file)
    frame_idx =-1
    gt_id=[]
    gt_bbox=[]
    for line in file:
        gt_info = line.split()
        if frame_idx == gt_info:
            gt_id.append(gt_info[1])
        else:
            gt_id.clear()
            gt_bbox.clear()
            if gt_info[1] != -1:
                gt_id.append(gt_info[1])
                gt_bbox = gt_info[10:17]
            frame_idx+=1
#                 gt_bbox.append(gt_info[10])
#                 gt_bbox.append(gt_info[11])
#                 gt_bbox.append(gt_info[12])
                
                
            


In [24]:
def tracking_vis(idx,result,gt=None): 
    vis = open3d.visualization.Visualizer()
    vis.create_window()
    vis.get_render_option().point_size = 1.0
    vis.get_render_option().background_color = np.zeros(3)
    axis_pcd = open3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])
    vis.add_geometry(axis_pcd)
    pts = open3d.geometry.PointCloud()
    points =points_list[idx][:,:3].cpu().numpy()
    pts.points = open3d.utility.Vector3dVector(points)
    pts.colors = open3d.utility.Vector3dVector(np.ones((points.shape[0], 3)))
    vis.add_geometry(pts)
    for i in range(result.shape[0]):
        vis_result =[]
        vis_result[0:3] = result[i,3:6]
        vis_result[3:6] = result[i,0:3]
        vis_result.append(result[i,6])
        
        line_set, box3d = translate_boxes_to_open3d_instance(vis_result)
        line_set.paint_uniform_color((1,1,0))
        id_str = str(int(result[i,7]))
#         print(id_str)
#         print(id_str)
#         print(vis_result[0:3])
        text_pose=[]
        text_pose=vis_result[0:3]
        text_pose[2]+=0.7
        text = text_3d(id_str,pos=text_pose[0:3])
    
        vis.add_geometry(text)
        vis.add_geometry(line_set)
        
#         o3d.visualization.draw_geometries([pcd, chessboard_coord])

            # if score is not None:
#     corners = box3d.get_box_points()
#     vis.add_3d_label(corners[5], '%.2f' % 0.1)
    
    vis.run()
    vis.destroy_window()

In [25]:
ID_start = 1
total_time = 0.0
tracker = AB3DMOT(cfg, "Car", ID_init=ID_start)

In [29]:
len(pred_det_list)
print(len(pred_det))

15


In [30]:
results_list=[]
for idx,pred_det in enumerate(pred_det_list):
    since = time.time()
    print(pred_det)
    print("======")
    results, affi = tracker.track(pred_det)
#     print(results)
    total_time += time.time() - since
    results = np.squeeze(results)
    results_list.append(results.tolist())
    
#     tracking_vis(idx,results)
#     if idx==1:
#         break
# h,w,l,x,y,z,theta, ID, other info, confidence



[[10.006596565246582, -2.557927370071411, -0.7775534391403198, 3.5052907466888428, 1.57882559299469, 1.5729707479476929, 6.291097164154053], [15.596556663513184, -2.4330854415893555, -0.7173831462860107, 4.279359340667725, 1.6089285612106323, 1.5292998552322388, 6.313475608825684], [21.633041381835938, -2.639524221420288, -0.68488609790802, 3.966742992401123, 1.6372156143188477, 1.6068859100341797, 6.2819905281066895], [26.478458404541016, -2.6406657695770264, -0.7155252695083618, 3.617187023162842, 1.590949535369873, 1.5038537979125977, 6.263675689697266], [10.807343482971191, 11.274028778076172, -1.1349283456802368, 4.624083518981934, 1.6080471277236938, 1.4904303550720215, 3.144822597503662], [4.540949821472168, 4.2641921043396, -0.9534140825271606, 3.9765872955322266, 1.5981085300445557, 1.516659140586853, 6.833230018615723], [12.148079872131348, 17.668697357177734, -1.0292304754257202, 3.8231468200683594, 1.5525970458984375, 1.42881441116333, 4.726099014282227], [36.99253845214844

[[11.8582181930542, -2.483839511871338, -0.7392986416816711, 4.3417229652404785, 1.6376192569732666, 1.5460703372955322, 6.2992424964904785], [18.03058624267578, -2.601026773452759, -0.7287112474441528, 4.078680038452148, 1.6193499565124512, 1.5760341882705688, 6.2785964012146], [6.317962646484375, -2.5404088497161865, -0.8149439692497253, 3.499987840652466, 1.5942753553390503, 1.5819640159606934, 6.26830530166626], [15.946941375732422, 4.485687255859375, -0.8514910936355591, 4.170212268829346, 1.6280725002288818, 1.5264105796813965, 2.4214694499969482], [29.708293914794922, 5.033243179321289, -0.7218897938728333, 3.6111958026885986, 1.6070109605789185, 1.46878182888031, 6.965766429901123], [22.713972091674805, -2.627891778945923, -0.7105202078819275, 3.5721447467803955, 1.5742357969284058, 1.486578345298767, 6.263381481170654], [1.1011943817138672, 4.445333957672119, -1.0241482257843018, 4.116069316864014, 1.6544095277786255, 1.4976001977920532, 6.917250156402588], [33.27935028076172,

[[8.198762893676758, -2.4259934425354004, -0.7938523888587952, 4.295497894287109, 1.6397922039031982, 1.5540552139282227, 6.300114154815674], [12.215015411376953, 4.541205883026123, -0.9041022062301636, 4.14319372177124, 1.5604734420776367, 1.5521891117095947, 2.4463818073272705], [25.893779754638672, 5.166591644287109, -0.7998384237289429, 3.6401565074920654, 1.5814164876937866, 1.4471299648284912, 7.051781177520752], [14.283377647399902, -2.586099624633789, -0.7496344447135925, 4.058073043823242, 1.6043545007705688, 1.6050341129302979, 6.265732288360596], [14.331263542175293, 11.294022560119629, -1.259408712387085, 3.6920294761657715, 1.5096560716629028, 1.3824955224990845, 3.1462600231170654], [2.6390628814697266, -2.5048580169677734, -0.8237709403038025, 3.6309385299682617, 1.5732076168060303, 1.5792808532714844, 6.288398742675781], [19.11469841003418, -2.5701231956481934, -0.7548667192459106, 3.7918498516082764, 1.5936251878738403, 1.5418224334716797, 6.284758567810059], [35.11203

[[4.500166893005371, -2.4225335121154785, -0.8556621670722961, 4.362045764923096, 1.6416305303573608, 1.564184308052063, 6.304688453674316], [8.5374174118042, 4.523931503295898, -0.9238385558128357, 4.173455238342285, 1.5891560316085815, 1.5667356252670288, 2.4388644695281982], [10.767097473144531, 11.316658020019531, -1.2784852981567383, 4.005283832550049, 1.5852463245391846, 1.4111076593399048, 3.143608808517456], [10.577207565307617, -2.581867218017578, -0.8242824077606201, 4.035874366760254, 1.6498932838439941, 1.591516137123108, 6.269339561462402], [22.208837509155273, 5.237802028656006, -0.8690766096115112, 3.660996913909912, 1.582657814025879, 1.4906765222549438, 7.037549018859863], [25.917116165161133, -2.4765408039093018, -0.5162692666053772, 4.482358932495117, 1.6877386569976807, 1.9575492143630981, 6.322849750518799], [27.644485473632812, 4.1109619140625, -0.8403315544128418, 3.745725154876709, 1.6824284791946411, 1.51426100730896, 1.4340825080871582], [15.39975643157959, -2

[[7.670662879943848, -2.5549042224884033, -0.8464024066925049, 4.057577133178711, 1.6708691120147705, 1.5746561288833618, 6.277729034423828], [19.304563522338867, 5.229661464691162, -0.8987764716148376, 3.5911056995391846, 1.593868374824524, 1.515021800994873, 7.043504238128662], [12.32140827178955, -2.55433988571167, -0.8770791888237, 3.517076015472412, 1.5893335342407227, 1.505357027053833, 6.275359630584717], [5.642990589141846, 4.531718730926514, -0.9541814923286438, 4.145361423492432, 1.6051658391952515, 1.550357699394226, 2.4434704780578613], [28.018938064575195, 11.55899715423584, -1.0678187608718872, 4.106539249420166, 1.5724103450775146, 1.4647666215896606, 3.1582906246185303], [7.813457012176514, 11.291654586791992, -1.2284917831420898, 3.92561936378479, 1.5602259635925293, 1.4336163997650146, 3.141918182373047], [24.645824432373047, 4.064408302307129, -0.8691022396087646, 3.5587923526763916, 1.6672505140304565, 1.516202449798584, 1.5129348039627075], [28.680828094482422, -2.

In [20]:
results_list


[[[4.114211082458496,
   1.69847571849823,
   1.5118030309677124,
   20.31446075439453,
   9.956417083740234,
   -0.9597275257110596,
   4.4708356857299805,
   15.0],
  [3.8031039237976074,
   1.6052131652832031,
   1.5266261100769043,
   53.40877151489258,
   -2.2582662105560303,
   -0.4052372872829437,
   6.113851547241211,
   14.0],
  [3.5269408226013184,
   1.637938141822815,
   1.5939960479736328,
   -0.0642298236489296,
   4.423957824707031,
   -0.9040671586990356,
   1.1798677444458008,
   13.0],
  [3.8221514225006104,
   1.5555840730667114,
   1.4111977815628052,
   33.38860321044922,
   5.170422554016113,
   -0.5927358865737915,
   7.062859535217285,
   12.0],
  [3.560187578201294,
   1.681391716003418,
   1.4811880588531494,
   38.608192443847656,
   3.834287166595459,
   -0.5971518754959106,
   1.4753142595291138,
   11.0],
  [3.7752509117126465,
   1.5860416889190674,
   1.4873989820480347,
   19.90793800354004,
   4.285452842712402,
   -0.7593076229095459,
   2.45434045791

In [21]:
import pickle

In [22]:
import pickle

with open('result', 'wb') as fp:
    pickle.dump(results_list, fp)

In [42]:
with open ('result', 'rb') as fp:
    itemlist = pickle.load(fp)
itemlist  

[[[3.962726174759007,
   1.579203916454166,
   1.4194181420620244,
   4.645525402061284,
   16.991736357996338,
   -1.1585760266997247,
   -1.5739903619352373,
   36.0],
  [4.184026572999074,
   1.5972796777417777,
   1.459988492941819,
   26.715508734312884,
   11.58467318421436,
   -1.0552303238061338,
   -3.1381214117913845,
   35.0],
  [4.000991640871895,
   1.6056225091058625,
   1.523626510140715,
   -0.38265262015635987,
   16.83576718206442,
   -1.0715846415102912,
   -1.4984638360463511,
   34.0],
  [3.3731156430287994,
   1.5209300439379532,
   1.482193269522767,
   31.131779505187783,
   11.529223538289175,
   -1.0780190007296802,
   -3.117324228259701,
   32.0],
  [3.6689014343089617,
   1.659368981755476,
   1.473365254392922,
   38.70962173498416,
   3.9474871179481257,
   -0.6210406159883584,
   1.2784429310461811,
   31.0],
  [4.192447638339086,
   1.6322665288557074,
   1.4278260970724859,
   2.1894431752791785,
   17.265592431731093,
   -1.1007543934280983,
   -1.6755

In [32]:
# with open("./results.txt", "rb") as lf:
#     readList = pickle.load(lf)
#     print(readList)
    
# with open(filePath, 'rb') as lf:
#     readList = pickle.load(lf)
#     print(readList)

EOFError: Ran out of input

In [1]:
import motmetrics as mm
import numpy as np




In [2]:
# Create an accumulator that will be updated during each frame
acc = mm.MOTAccumulator(auto_id=True)

# Call update once for per frame. For now, assume distances between
# frame objects / hypotheses are given.
acc.update(
    [1, 2],                     # Ground truth objects in this frame
    [1, 2, 3],                  # Detector hypotheses in this frame
    [
        [0.1, np.nan, 0.3],     # Distances from object 1 to hypotheses 1, 2, 3
        [0.5,  0.2,   0.3]      # Distances from object 2 to hypotheses 1, 2, 3
    ]
)

0

In [3]:
frameid = acc.update(
    [1, 2],
    [1],
    [
        [0.2],
        [0.4]
    ]
)


In [4]:
# frameid = acc.update(
#     [1, 2],
#     [1, 3],
#     [
#         [0.6, 0.2],
#         [0.1, 0.6]
#     ]
# )

In [5]:
mh = mm.metrics.create()
summary = mh.compute(acc, metrics=['num_frames', 'mota', 'motp'], name='acc')
print(summary)

     num_frames  mota      motp
acc           2   0.5  0.166667


In [6]:
strsummary = mm.io.render_summary(
    summary,
    formatters={'mota' : '{:.2%}'.format},
    namemap={'mota': 'MOTA', 'motp' : 'MOTP'}
)
print(strsummary)

     num_frames   MOTA      MOTP
acc           2 50.00%  0.166667


In [7]:
# fig = visualize_pts(points_list[0][:,:3].cpu().numpy())
# fig = draw_multi_grid_range(fig, bv_range=(0, -40, 80, 40))
# # ref_corners3d = boxes_to_corners_3d([1,1,1,1,1,1,1])
# fig = draw_corners3d([1.,1.,1.,1.,1.,1.,1.], fig=fig, color=(0, 1, 0), cls=[91], max_num=100)

In [20]:
# !pip install image
# sudo fc-cache -f -v

In [21]:

chessboard_coord = o3d.geometry.TriangleMesh.create_coordinate_frame(
        size=0.02, origin=[0, 0, 0])
# pcd_10 = text_3d('Test-10mm', pos=[0, 0, 0.01], font_size=10, density=10)
# pcd_20 = text_3d('Test-20mm', pos=[0, 0, 0], font_size=20, density=2)
pcd = text_3d("test",pos=[0,0,0])
o3d.visualization.draw_geometries([pcd, chessboard_coord])
